In [17]:
import pandas as pd

import pyodbc
import sqlalchemy
import urllib

import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [18]:
# Google API Check Function
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [19]:
# Google Sheet Data Pull Function
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [20]:
# Connect to MS SQL Server
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=winwin; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect(conn_string);
crs = conn.cursor()
conn.autocommit = True

In [21]:
# MS SQL Execute Query
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [22]:
# MS SQL Read Query
def read_query (conn, query):
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [23]:
# Create Schema
# query = 'CREATE SCHEMA Orders'
# execute_query(conn, query)

In [24]:
# # Create Table
# query = """CREATE TABLE Orders.BellonaAll (InvoiceId NVARCHAR (50),
#    										  OrderId NVARCHAR (50),
#    									  	  TripId NVARCHAR (50),
#    										  OrderDate DATE,
#    										  Store NVARCHAR (50),
#    										  Vendor NVARCHAR (50),
#    										  Item NVARCHAR (MAX),
#    										  SKU NVARCHAR (50),
#    										  New_Item NVARCHAR (MAX),
#    										  Shiped INT,
#    										  UnitPrice NVARCHAR (50),
#    										  Ordered INT,
#    										  FreightPrice NVARCHAR (50),
#    										  TotalPrice NVARCHAR (50),
#    										  ExtendedPrice NVARCHAR (50),
#    										  Merchandise NVARCHAR (50),
#    										  Freight_Ocean NVARCHAR (50),
#    										  Fuel_Ship NVARCHAR (50),
#    										  Tax NVARCHAR (50),
#    										  Discount NVARCHAR (50),
#    										  AllTotal NVARCHAR (50));"""
# execute_query(conn, query)

In [ ]:
# Google Sheet Data Pull
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = 'sfafdsvawervrcer-xrcarareyIds'

#Pulls data from the entire spreadsheet tab.
#DATA_TO_PULL = 'BellonaAll'

#Pulls data only from the specified range of cells.
DATA_TO_PULL = 'serfervfwrvwerv!A1:U10000'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns=data[0])
print(df.shape)
df.head(5)

In [26]:
# Use winwin database
query = 'USE winwin'
execute_query(conn, query)

Query Succeessful!


In [27]:
# Truncate the table
query = 'TRUNCATE TABLE Orders.BellonaAll'
execute_query(conn, query)

Query Succeessful!


In [28]:
# Read the Empty Table
pd.read_sql("SELECT * FROM Orders.BellonaAll", con = conn)

,InvoiceId,OrderId,TripId,OrderDate,Store,Vendor,Item,SKU,New_Item,Shiped,...,Ordered,FreightPrice,TotalPrice,ExtendedPrice,Merchandise,Freight_Ocean,Fuel_Ship,Tax,Discount,AllTotal


In [29]:
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
        crs.execute(""" INSERT INTO Orders.BellonaAll
                                (InvoiceId, OrderId, TripId, OrderDate, Store, Vendor,
                                Item, SKU, New_Item, Shiped, UnitPrice, Ordered, FreightPrice,
                                TotalPrice, ExtendedPrice, Merchandise, Freight_Ocean,
                                Fuel_Ship, Tax, Discount, AllTotal)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) """,
                                row.InvoiceId, row.OrderId, row.TripId, row.OrderDate, row.Store, row.Vendor,
                                row.Item, row.SKU, row.New_Item, row.Shiped, row.UnitPrice, row.Ordered,
                                row.FreightPrice, row.TotalPrice, row.ExtendedPrice, row.Merchandise,
                                row.Freight_Ocean, row.Fuel_Ship, row.Tax, row.Discount, row.AllTotal)
conn.commit()
crs.close()

In [ ]:
# Read the Full Table
df = pd.read_sql("SELECT * FROM Orders.BellonaAll", con = conn)
print(df.shape)
df.head(5)